### Import the libraries

In [ ]:
from astropy.time import Time
from astroquery.jplhorizons import Horizons

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import time

### Read the database

In [ ]:
csv_input='skm-obs.dat'
df=pd.read_fwf(csv_input,colspecs="infer", engine='python', header=None, dtype = {11: 'string'})

In [ ]:
df = df.rename(columns = {0: 'Source', 1: 'Frame', 2: 'jd', 3: 'Filter', 4: 'Exptime', 5: 'RAdeg', 6: 'DEdeg', 7: 'm', 8: 'err_m', 
              9: 'petromag', 10: 'e_petromag', 11: 'idAsteroid', 12: 'desig', 13: 'DynClass', 14: 'RArate' , 15: 'DErate', 16: 'Vmag'})

In [ ]:
df = df[['idAsteroid', 'desig', 'jd', 'm', 'err_m', 'Filter']]

In [ ]:
df = df.sort_values(by='idAsteroid')

In [ ]:
# Some asteroids have only designation (not number), so we merge those columns to adapt it to our style.
df['idAsteroid'] = df['idAsteroid'].fillna(df.pop('desig'))

### Avoid repetition

Sometimes, during the Horizons query, some problems may appear. For example, bad internet connection. If we restart the code,
we don't want to repeat the process for asteroids that already have a .csv file. This part of the code prevent that.

In [ ]:
# # Specify the folder path where your CSV files are located
folder_path = '/home/milagros/Documents/SkyMapper-files'

# # Initialize an empty list to store the formatted asteroid numbers
formatted_asteroid_numbers = []

# # Define a regular expression pattern to match the filenames
pattern = r'SkyMapper-asteroid(\d+)\.csv'

# # List all files in the folder
files = os.listdir(folder_path)

# # Iterate through the files and extract the numbers
for filename in files:
    match = re.match(pattern, filename)
    if match:
        number = (match.group(1))
       
        formatted_asteroid_numbers.append(number)

In [ ]:
df = df[~df['idAsteroid'].isin(formatted_asteroid_numbers)]

### Calculations

In [ ]:
# We need to perform some calculations to obtain the desired format for the table.

# Compute flux
flux = 10**(-0.4*df.m)
flux = flux.to_numpy()
df.insert(4, "flux", flux, True)
e_flux = 0.4*10**(-0.4*df.m)*df.err_m
df.insert(5, "err_flux", e_flux, True)
# ----------------------------------------------------------------------------------------------------------------------
#Add constant columns
df.insert(5, 'Source', 'SkyMapper')
df.insert(6, 'TypePhotometry', 'relative')
df.insert(7, 'LTcorrected', '0')
# ----------------------------------------------------------------------------------------------------------------------
#Add ephemeris columns
df.insert(8, 'ElongFlag', '')
df.insert(9, 'alpha', '')
df.insert(10, 'r', '')
df.insert(11, 'delta', '')
df.insert(12, 'x_obs', '')
df.insert(13, 'y_obs', '')
df.insert(14, 'z_obs', '')
df.insert(15, 'x_sun', '')
df.insert(16, 'y_sun', '')
df.insert(17, 'z_sun', '')
df.insert(18, 'LT_obs', '')

### Splitting dataframe

We have a dataframe with all the asteroids. For the ephemeris step, we need to proceed asteroid by asteroid.

In [ ]:
sample = df
ids_sample = sample['idAsteroid'].drop_duplicates()
ids_sample = ids_sample.to_numpy()
len = ids_sample.size

In [ ]:
# Function to split DataFrame based on the changing value in the 'target_column'
def split_dataframe_on_value_change(df, column_name):
    df_list = []
    prev_value = None
    temp_df = None

    for index, row in df.iterrows():
        current_value = row[column_name]

        if prev_value is None or current_value != prev_value:
            if temp_df is not None:
                df_list.append(temp_df)
            temp_df = pd.DataFrame(columns=df.columns)

        temp_df = pd.concat([temp_df, row.to_frame().T], ignore_index=True)
        prev_value = current_value

    if temp_df is not None:
        df_list.append(temp_df)

    return df_list

In [ ]:
# Split the DataFrame based on the changing value in the 'target_column'
resulting_dataframes = split_dataframe_on_value_change(sample, 'idAsteroid')

### Ephemeris

In [ ]:
# Ephemeris
for idx, df in enumerate(resulting_dataframes):
    ids = df['idAsteroid'].drop_duplicates().to_numpy()[0]
    jd = df['jd'].to_numpy()

    # If the list we pass to Horizons contains more than 50 elements, it throws an error.
    if (jd.size <= 50):
    
        #Ephemeris query
        e_obj = Horizons(id=ids, location='Q55@399', epochs=jd, id_type='smallbody') # Julian date in UTC for ephemerides
        eph = e_obj.ephemerides()
        elong_flag = eph.columns['elongFlag']
        alpha = eph.columns['alpha']
        delta = eph.columns['delta']
        r = eph.columns['r']
        
        #Vectors query
        jd_tdb = Time(jd.tolist(), format='jd', scale='utc').tdb.value
        v_obj = Horizons(id=ids, location='Q55@399', epochs=jd_tdb, id_type='smallbody') # Julian date in TDB for vectors
        vec = v_obj.vectors()
        x_obs = vec.columns['x']
        y_obs = vec.columns['y']
        z_obs = vec.columns['z']
        LT_obs = vec.columns['lighttime']
    
        #Query Sun-----------------------------------------
        obj_sun = Horizons(id=ids, location='500@10',
                epochs=jd_tdb, id_type='smallbody')
        vec_sun = obj_sun.vectors()
        x_sun = vec_sun.columns['x']
        y_sun = vec_sun.columns['y']
        z_sun = vec_sun.columns['z']  
    
        #Put in dataframe
        df['ElongFlag'] = elong_flag
        df['alpha'] = alpha
        df['r'] = r
        df['delta'] = delta
        df['x_obs'] = x_obs
        df['y_obs'] = y_obs
        df['z_obs'] = z_obs
        df['x_sun'] = x_sun
        df['y_sun'] = y_sun
        df['z_sun'] = z_sun
        df['LT_obs'] = LT_obs
        
        m_red = df['m'] - 5*np.log10(df['r']*df['delta'])
        df.insert(6, "m_red", m_red, True)
        df = df[['idAsteroid', 'jd', 'm', 'err_m', 'flux', 'err_flux', 'm_red', 'Filter', 'alpha', 'delta', 'r', 'ElongFlag', 'Source', 'TypePhotometry', 'LTcorrected','x_obs', 'y_obs', 'z_obs', 'x_sun', 'y_sun', 'z_sun', 'LT_obs']]
        df.to_csv('/home/milagros/Documents/SkyMapper-files/SkyMapper-asteroid'+str(ids)+'.csv', index=False) 
        
        del obj
        del eph
        del obj_sun
        del vec_sun
        gc.collect()
        print(ids, 'small done')

    # To process lists of more than 50 elements, we split the dataframe and work part by part. At the end we merge everything into a single table.
    if (jd.size > 50):
        split_dataframes = np.array_split(df, 5)
        calculated_dataframes = []
        for i in split_dataframes

            #Ephemeris query
            e_obj = Horizons(id=ids, location='Q55@399', epochs=i.jd.to_numpy(), id_type='smallbody')
            eph = e_obj.ephemerides()
            elong_flag = eph.columns['elongFlag']
            alpha = eph.columns['alpha']
            delta = eph.columns['delta']
            r = eph.columns['r']
            
            #Vectors query
            jd_tdb = Time(i.jd.tolist(), format='jd', scale='utc').tdb.value
            v_obj = Horizons(id=ids, location='Q55@399', epochs=jd_tdb, id_type='smallbody')
            vec = v_obj.vectors()
            x_obs = vec.columns['x']
            y_obs = vec.columns['y']
            z_obs = vec.columns['z']
            LT_obs = vec.columns['lighttime']
        
            #Query Sun-----------------------------------------
            obj_sun = Horizons(id=ids, location='500@10', epochs=jd_tdb, id_type='smallbody')
            vec_sun = obj_sun.vectors()
            x_sun = vec_sun.columns['x']
            y_sun = vec_sun.columns['y']
            z_sun = vec_sun.columns['z']  
        
            #Put in dataframe
            i['ElongFlag'] = elong_flag
            i['alpha'] = alpha
            i['r'] = r
            i['delta'] = delta
            i['x_obs'] = x_obs
            i['y_obs'] = y_obs
            i['z_obs'] = z_obs
            i['x_sun'] = x_sun
            i['y_sun'] = y_sun
            i['z_sun'] = z_sun
            i['LT_obs'] = LT_obs
            
            m_red = i['m'] - 5*np.log10(i['r']*i['delta'])
            i['m_red'] = m_red
            
            calculated_dataframes.append(i)
            del obj
            del eph
            del obj_sun
            del vec_sun
            gc.collect()
            
        merged_df = pd.concat(calculated_dataframes, ignore_index=True)
        merged_df = merged_df[['idAsteroid', 'jd', 'm', 'err_m', 'flux', 'err_flux', 'm_red', 'Filter', 'alpha', 'delta', 'r', 'ElongFlag', 'Source', 'TypePhotometry', 'LTcorrected','x_obs', 'y_obs', 'z_obs', 'x_sun', 'y_sun', 'z_sun', 'LT_obs']]
        merged_df['ElongFlag'] = merged_df['ElongFlag'].to_numpy()
        merged_df['alpha'] = merged_df['alpha'].to_numpy()
        merged_df['r'] = merged_df['r'].to_numpy()
        merged_df['delta'] = merged_df['delta'].to_numpy()
        merged_df['m_red'] = merged_df['m_red'].to_numpy()
        merged_df['x_obs'] = merged_df['x_obs'].to_numpy()
        merged_df['y_obs'] = merged_df['y_obs'].to_numpy()
        merged_df['z_obs'] = merged_df['z_obs'].to_numpy()
        merged_df['x_sun'] = merged_df['x_sun'].to_numpy()
        merged_df['y_sun'] = merged_df['y_sun'].to_numpy()
        merged_df['z_sun'] = merged_df['z_sun'].to_numpy()
        merged_df['LT_obs'] = merged_df['LT_obs'].to_numpy()
        
        merged_df.to_csv('/home/milagros/Documents/SkyMapper-files/SkyMapper-asteroid'+str(ids)+'.csv', index=False)
        print(ids, 'big done')
    

The end :)